In [2]:
import gensim
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, punkt
from gensim.models import KeyedVectors

#setting up libraries
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time
import spacy


In [3]:
df_big = pd.read_pickle("/home/seh6fy/git/publicrd/data/prd/Paper/FR_meta_and_final_tokens_23DEC21.pkl")
df = pd.read_csv("/home/seh6fy/git/publicrd/data/prd/Digital_abstract_labelled/labelled_abstracts.csv")

In [4]:
df_big = df_big[~df_big.ABSTRACT.isnull()]

In [5]:
df = df[~df.label.isnull()]
df_big = df_big[~df_big.ABSTRACT.isnull()]

In [6]:
df_nsf = df.loc[df['label'] == 'Big-data']
df_nsf = df_nsf.iloc[0:384,]

In [7]:
df_nasa = df.loc[df['label'] == 'Non Big-data']
df_nasa = df_nasa.iloc[0:384,]

In [8]:
frames = [df_nsf, df_nasa]
df_split = pd.concat(frames)

In [9]:
#creates a dataframe with the training and testing data 
both = [df_split, df_big]
new_df= pd.concat(both)

In [10]:
new_df= new_df.reset_index(level=None, drop=True, inplace=False, col_level=0, col_fill='')

In [11]:
data = new_df['ABSTRACT']

In [12]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
tagged_df = pd.DataFrame(tagged_data)

In [13]:
tagged_df

,words,tags
0,"[ï, », ¿, description, (, provided, by, applic...",[0]
1,"[project, summary/abstractthe, establishment, ...",[1]
2,"[abstractthis, ahrq, k08, project, employs, bi...",[2]
3,"[project, summary, â€, “, epidemiology, and, c...",[3]
4,"[description, (, provided, by, applicant, ), :...",[4]
...,...,...
1144632,"[circadian, rhythms, are, fundamental, endogen...",[1144632]
1144633,"[recent, research, suggests, that, learning, a...",[1144633]
1144634,"[the, covid-19, pandemic, has, caused, unprece...",[1144634]
1144635,"[subduction, zones, –, locations, where, one, ...",[1144635]


In [14]:
new_df['words'] = tagged_df['words']
new_df['tags'] = tagged_df['tags']
new_df.head()

,Unnamed: 0,Duplicates,index,PROJECT_ID,ABSTRACT,Is it related to Big-Data,Expert name,label,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens,words,tags
0,0.0,0.0,979.0,950056,ï»¿ DESCRIPTION (provided by applicant): Adv...,1,Kristian,Big-data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ï, », ¿, description, (, provided, by, applic...",[0]
1,1.0,0.0,816.0,1178172,Project Summary/AbstractThe establishment of t...,1,Kristian,Big-data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[project, summary/abstractthe, establishment, ...",[1]
2,7.0,0.0,333.0,1163576,AbstractThis AHRQ K08 project employs big data...,1,Kristian,Big-data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[abstractthis, ahrq, k08, project, employs, bi...",[2]
3,14.0,0.0,741.0,1178158,PROJECT SUMMARY â€“ EPIDEMIOLOGY AND CLINICAL ...,1,Kristian,Big-data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[project, summary, â€, “, epidemiology, and, c...",[3]
4,22.0,0.0,549.0,620880,DESCRIPTION (provided by applicant): This appl...,1,Kristian,Big-data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[description, (, provided, by, applicant, ), :...",[4]


In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, epochs=15) #min_count=2 - min number of times it shows up
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
model.save("d2v.model")

In [ ]:
model = Doc2Vec.load("d2v.model")

In [ ]:
vector = []
for i in range(0, len(df_split)):
    vectorthis = (model.dv[df_split['tags'][i]]).tolist()
    vector.append(vectorthis)

In [ ]:
mat = []
for i in vector:
    for x in i:
        mat.append(x)

In [ ]:
training = pd.DataFrame(mat)

In [ ]:
trainingnp = training.to_numpy()

In [ ]:
training_data =  new_df[~new_df.label.isnull()]
testing_data = new_df.drop(training_data.index)

In [ ]:
train_id = training_data.index
test_id = testing_data.index

In [ ]:
x_train = trainingnp[train_id]
x_test = trainingnp[test_id]

In [ ]:
df_split['index'] = df_split.index
y_test = df_split.loc[df_split["index"].isin(test_id)]
y_test = y_test['label']

In [ ]:
df_split['index'] = df_split.index
y_train = df_split.loc[df_split["index"].isin(train_id)]
y_train = y_train['label']

In [ ]:
import math

In [ ]:
neighbors = int(math.sqrt(len(df_split.index)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 15 )
classifier.fit(x_train, y_train)

In [ ]:
# from sklearn.svm import SVC
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# clf = make_pipeline(StandardScaler(), SVC(gamma='auto', ))
# clf.fit(x_train, y_train)


In [ ]:
y_pred = classifier.predict(x_test)

In [ ]:
# y_pred = clf.predict(x_test)

In [ ]:
import scipy.sparse

In [ ]:
predictions = df_split.loc[test_id,]
predictions['PREDICTIONS'] = y_pred

In [ ]:
predictions = predictions[["index","label", "PREDICTIONS","ABSTRACT"]]

In [ ]:
big_data = predictions.loc[predictions['PREDICTIONS'] == 'Big-data']

In [ ]:
merged_inner = pd.merge(left=big_data, right=new_df, left_on='ABSTRACT', right_on='ABSTRACT')

In [ ]:
merged_inner = merged_inner[[ "PREDICTIONS","ABSTRACT", "PROJECT_ID","PROJECT_TERMS","PROJECT_TITLE","DEPARTMENT","AGENCY","PROJECT_NUMBER","PROJECT_START_DATE","PROJECT_END_DATE","FY_TOTAL_COST"]]

In [ ]:
merged_inner.to_csv(r'doc2vec_dataframe.csv')

In [1180]:
# predictions['PREDICTIONS'].value_counts()

In [1181]:
# predictions['label'].value_counts()

In [1182]:
# df_split['label'].value_counts()

In [1183]:
# predictions.to_csv('precision.csv')

In [1184]:
predictions['PREDICTIONS'].value_counts()

Non Big-data    65
Big-data        50
Name: PREDICTIONS, dtype: int64

In [1185]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[30 31]
 [20 34]]
              precision    recall  f1-score   support

    Big-data       0.60      0.49      0.54        61
Non Big-data       0.52      0.63      0.57        54

    accuracy                           0.56       115
   macro avg       0.56      0.56      0.56       115
weighted avg       0.56      0.56      0.56       115



##### error = []

# Calculating error for K values between 1 and 40
for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    pred_i = knn.predict(x_test)
    error.append(np.mean(pred_i != y_test))